# Quantum Information - Code Laboratory

**Section 9: Quantum Information** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Class/Function | Signature | Returns | Use When |
|----------------|-----------|---------|----------|
| `Clifford(circuit)` | `Clifford(data)` | `Clifford` | Efficient Clifford simulation |
| `Operator(circuit)` | `Operator(data)` | `Operator` | Full unitary matrix |
| `Statevector.from_label()` | `from_label(label)` | `Statevector` | Create pure state |
| `DensityMatrix()` | `DensityMatrix(data)` | `DensityMatrix` | Mixed states |
| `state_fidelity()` | `state_fidelity(s1, s2)` | `float [0,1]` | Compare states |
| `process_fidelity()` | `process_fidelity(c1, c2)` | `float [0,1]` | Compare processes |
| `Operator.equiv()` | `op1.equiv(op2)` | `bool` | Phase-invariant comparison |

---

In [ ]:
"""
Qiskit Code Laboratory - Quantum Information
=============================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import (
    Clifford, Operator, Statevector, DensityMatrix,
    state_fidelity, process_fidelity, average_gate_fidelity,
    random_clifford, Kraus, SuperOp, Choi, partial_trace
)
from qiskit.circuit.library import HGate, XGate, ZGate, SGate, TGate

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_state(qc, label=""):
    """Display statevector with optional label."""
    sv = Statevector.from_instruction(qc)
    print(f"{label}: {sv}")
    return sv

def compare_operators(op1, op2, name1="Op1", name2="Op2"):
    """Compare two operators and show equivalence."""
    print(f"{name1} == {name2} (strict): {op1 == op2}")
    print(f"{name1}.equiv({name2}) (phase-invariant): {op1.equiv(op2)}")
    return op1.equiv(op2)

print("✅ Environment ready - Quantum Information Code Laboratory")

---

## `Clifford` Class

### Signature
```python
Clifford(data, validate=True)
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `data` | `QuantumCircuit`, `Clifford`, `Pauli` | Yes | - | Clifford circuit/object |
| `validate` | `bool` | No | `True` | Validate input is Clifford |

### Returns
`Clifford` - Clifford operator object

### Raises
- `QiskitError`: If circuit contains non-Clifford gates (like T)

### See Also
- README Section: [Clifford Circuits](./README.md#-topic-91-clifford-circuits)

In [ ]:
# ============================================================
# Clifford() - BASIC USAGE
# ============================================================

# Create a Clifford circuit (H, S, CNOT only!)
qc = QuantumCircuit(2)
qc.h(0)      # ✅ Clifford
qc.s(0)      # ✅ Clifford  
qc.cx(0, 1)  # ✅ Clifford

print("Clifford circuit:")
print(qc.draw())

# Create Clifford object
cliff = Clifford(qc)
print(f"\n✅ Clifford created successfully!")
print(f"Number of qubits: {cliff.num_qubits}")

# Verify it works
assert cliff.num_qubits == 2, "Should be 2 qubits"
print("✅ Basic Clifford usage verified")

In [ ]:
# ============================================================
# Clifford() - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Composition of Cliffords
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.cx(0, 1)
cliff1 = Clifford(qc1)

qc2 = QuantumCircuit(2)
qc2.s(0)
qc2.s(1)
cliff2 = Clifford(qc2)

composed = cliff1.compose(cliff2)
print(f"Composed Clifford: {composed.num_qubits} qubits")

# Variation 2: Random Clifford
random_cliff = random_clifford(2, seed=42)
print(f"\nRandom 2-qubit Clifford created")

# Variation 3: Convert back to circuit
recovered_circuit = cliff1.to_circuit()
print(f"\nConverted back to circuit:")
print(recovered_circuit.draw())

print("\n✅ All Clifford variations demonstrated")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: T gate is NOT Clifford!
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: T gate is NOT Clifford!")
print("=" * 50)

# ❌ WRONG - T gate breaks Clifford
print("\n❌ WRONG: Using T gate in Clifford circuit")
qc_wrong = QuantumCircuit(1)
qc_wrong.h(0)
qc_wrong.t(0)  # T gate - NOT Clifford!
print(qc_wrong.draw())

try:
    cliff_wrong = Clifford(qc_wrong)
    print("Created Clifford (unexpected!)")
except Exception as e:
    print(f"✅ Error caught: {type(e).__name__}")
    print("   T gate is NOT in the Clifford group!")

# ✅ CORRECT - S gate IS Clifford
print("\n✅ CORRECT: Using S gate (Clifford)")
qc_correct = QuantumCircuit(1)
qc_correct.h(0)
qc_correct.s(0)  # S gate - IS Clifford!
print(qc_correct.draw())

cliff_correct = Clifford(qc_correct)
print(f"✅ Clifford created successfully!")

print("\n💡 Remember: HSCP - No T! (See README for mnemonic)")

In [ ]:
# ============================================================
# Clifford() - VERIFICATION PATTERN
# ============================================================

def test_clifford():
    """Test suite for Clifford class."""
    
    # Test 1: Basic creation
    qc = QuantumCircuit(1)
    qc.h(0)
    cliff = Clifford(qc)
    assert cliff.num_qubits == 1, "Test 1 failed: wrong qubit count"
    
    # Test 2: Composition preserves qubits
    qc1 = QuantumCircuit(2)
    qc1.h(0)
    qc2 = QuantumCircuit(2)
    qc2.cx(0, 1)
    composed = Clifford(qc1).compose(Clifford(qc2))
    assert composed.num_qubits == 2, "Test 2 failed: composition wrong"
    
    # Test 3: All Clifford gates work
    qc_all = QuantumCircuit(2)
    qc_all.h(0)
    qc_all.s(0)
    qc_all.x(0)
    qc_all.y(1)
    qc_all.z(1)
    qc_all.cx(0, 1)
    cliff_all = Clifford(qc_all)
    assert cliff_all.num_qubits == 2, "Test 3 failed: not all Clifford gates"
    
    print("✅ All Clifford tests passed!")

test_clifford()

---

## `Operator` Class

### Signature
```python
Operator(data, input_dims=None, output_dims=None)
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `data` | `QuantumCircuit`, `Gate`, `ndarray` | Yes | - | Unitary to convert |
| `input_dims` | `tuple` | No | `None` | Input subsystem dimensions |
| `output_dims` | `tuple` | No | `None` | Output subsystem dimensions |

### Key Methods
| Method | Returns | Description |
|--------|---------|-------------|
| `.data` | `ndarray` | The unitary matrix |
| `.equiv(other)` | `bool` | Phase-invariant equivalence check |
| `.compose(other)` | `Operator` | Matrix multiplication (other first!) |
| `.tensor(other)` | `Operator` | Tensor product |

### See Also
- README Section: [Operator Class](./README.md#-topic-92-operator-class)

In [ ]:
# ============================================================
# Operator() - BASIC USAGE
# ============================================================

# Create Operator from circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

op = Operator(qc)

print("Operator from Bell circuit:")
print(f"Dimensions: {op.dim}")
print(f"Number of qubits: {op.num_qubits}")
print(f"\nMatrix (rounded):\n{np.round(op.data, 3)}")

# Verify dimensions
assert op.num_qubits == 2, "Should be 2 qubits"
assert op.data.shape == (4, 4), "Should be 4x4 matrix"
print("\n✅ Basic Operator usage verified")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: == vs .equiv()
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Using == instead of .equiv()")
print("=" * 50)

# Create two circuits equivalent up to global phase
qc1 = QuantumCircuit(1)
qc1.x(0)

qc2 = QuantumCircuit(1)
qc2.x(0)
qc2.global_phase = np.pi  # Add global phase

op1 = Operator(qc1)
op2 = Operator(qc2)

# ❌ WRONG - Using ==
print("\n❌ WRONG: Using == for comparison")
print(f"op1 == op2: {op1 == op2}")  # False!
print("   Returns False due to global phase difference")

# ✅ CORRECT - Using .equiv()
print("\n✅ CORRECT: Using .equiv() for comparison")
print(f"op1.equiv(op2): {op1.equiv(op2)}")  # True!
print("   Returns True (same physics, ignores global phase)")

print("\n💡 Remember: EQUIV for EQUIValent physics! (See README)")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Compose order confusion
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Compose order is counterintuitive!")
print("=" * 50)

h_op = Operator(HGate())
x_op = Operator(XGate())

# The circuit: H first, then X
qc = QuantumCircuit(1)
qc.h(0)
qc.x(0)
circuit_op = Operator(qc)
print("Circuit order: H → X")
print(qc.draw())

# ❌ WRONG thinking: h_op.compose(x_op) = H then X?
print("\n❌ WRONG thinking: h_op.compose(x_op) means H then X")
wrong_compose = h_op.compose(x_op)
print(f"h_op.compose(x_op) == circuit? {wrong_compose.equiv(circuit_op)}")

# ✅ CORRECT: x_op.compose(h_op) = H then X
print("\n✅ CORRECT: x_op.compose(h_op) means H then X")
correct_compose = x_op.compose(h_op)
print(f"x_op.compose(h_op) == circuit? {correct_compose.equiv(circuit_op)}")

print("\n💡 Remember: Compose = Right to Left (like matrix mult)")

In [ ]:
# ============================================================
# Operator() - VERIFICATION PATTERN
# ============================================================

def test_operator():
    """Test suite for Operator class."""
    
    # Test 1: X·X = I
    qc = QuantumCircuit(1)
    qc.x(0)
    qc.x(0)
    op_xx = Operator(qc)
    op_i = Operator(np.eye(2))
    assert op_xx.equiv(op_i), "Test 1 failed: X·X should equal I"
    
    # Test 2: HXH = Z
    h_op = Operator(HGate())
    x_op = Operator(XGate())
    z_op = Operator(ZGate())
    hxh = h_op.compose(x_op).compose(h_op)
    assert hxh.equiv(z_op), "Test 2 failed: HXH should equal Z"
    
    # Test 3: Equiv ignores global phase
    qc1 = QuantumCircuit(1)
    qc1.x(0)
    qc2 = QuantumCircuit(1)
    qc2.x(0)
    qc2.global_phase = np.pi/4
    assert Operator(qc1).equiv(Operator(qc2)), "Test 3 failed: equiv should ignore phase"
    
    print("✅ All Operator tests passed!")

test_operator()

---

## `Statevector` and `DensityMatrix`

### Statevector Signature
```python
Statevector(data, dims=None)
Statevector.from_label(label)
Statevector.from_instruction(instruction)
```

### DensityMatrix Signature
```python
DensityMatrix(data, dims=None)
DensityMatrix.from_label(label)
DensityMatrix.from_instruction(instruction)
```

### Key Difference
| Aspect | Statevector | DensityMatrix |
|--------|-------------|---------------|
| State Type | Pure only | Pure + Mixed |
| Representation | Vector | Matrix |
| Use Case | Ideal simulation | Noise simulation |
| Purity Check | Always 1 | Use `.purity()` |

### See Also
- README Section: [Statevector](./README.md#-topic-93-statevector) and [DensityMatrix](./README.md#-topic-935-densitymatrix)

In [ ]:
# ============================================================
# Statevector - BASIC USAGE
# ============================================================

# Create from label
sv_0 = Statevector.from_label('0')
sv_plus = Statevector.from_label('+')
sv_00 = Statevector.from_label('00')

print("Statevector from labels:")
print(f"|0⟩: {sv_0}")
print(f"|+⟩: {sv_plus}")
print(f"|00⟩: {sv_00}")

# Create from circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
bell_state = Statevector.from_instruction(qc)
print(f"\nBell state from circuit: {bell_state}")

# Get probabilities
probs = bell_state.probabilities()
print(f"Probabilities: {probs}")

assert np.isclose(probs[0], 0.5) and np.isclose(probs[3], 0.5), "Bell state should have 50/50"
print("\n✅ Statevector basic usage verified")

In [ ]:
# ============================================================
# Statevector - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Different label formats
print("[1] Different label formats:")
print("-" * 40)
sv_single = Statevector.from_label('1')       # Single qubit
sv_multi = Statevector.from_label('010')      # Multi-qubit
sv_special = Statevector.from_label('+-r')    # Special states (+, -, r, l)
print(f"|1⟩: {sv_single}")
print(f"|010⟩: {sv_multi}")
print(f"|+-r⟩: {sv_special}")

# Variation 2: Evolve through operator
print("\n[2] Evolve through operator:")
print("-" * 40)
sv = Statevector.from_label('0')
x_op = Operator(XGate())
sv_evolved = sv.evolve(x_op)
print(f"|0⟩ evolved by X: {sv_evolved}")

# Variation 3: Tensor product
print("\n[3] Tensor product:")
print("-" * 40)
sv_0 = Statevector.from_label('0')
sv_plus = Statevector.from_label('+')
sv_tensor = sv_0.tensor(sv_plus)
print(f"|0⟩ ⊗ |+⟩ = {sv_tensor}")

# Variation 4: Draw on Bloch sphere (single qubit)
print("\n[4] Probabilities and expectation values:")
print("-" * 40)
sv = Statevector.from_label('+')
print(f"|+⟩ probabilities: {sv.probabilities()}")
print(f"|+⟩ probabilities dict: {sv.probabilities_dict()}")

print("\n✅ Statevector variations demonstrated")

In [ ]:
# ============================================================
# Statevector - VERIFICATION PATTERN
# ============================================================

def test_statevector():
    """Test suite for Statevector class."""
    
    # Test 1: Correct dimensions
    sv = Statevector.from_label('00')
    assert sv.num_qubits == 2, "Test 1 failed: should have 2 qubits"
    assert len(sv.data) == 4, "Test 1 failed: should have 4 amplitudes"
    
    # Test 2: Normalization is preserved
    sv = Statevector.from_label('+')
    norm = np.sum(np.abs(sv.data)**2)
    assert np.isclose(norm, 1.0), "Test 2 failed: should be normalized"
    
    # Test 3: Evolve preserves normalization
    sv = Statevector.from_label('0')
    sv_evolved = sv.evolve(Operator(HGate()))
    norm = np.sum(np.abs(sv_evolved.data)**2)
    assert np.isclose(norm, 1.0), "Test 3 failed: evolve should preserve norm"
    
    # Test 4: Probabilities sum to 1
    sv = Statevector.from_label('+-')
    probs = sv.probabilities()
    assert np.isclose(sum(probs), 1.0), "Test 4 failed: probs should sum to 1"
    
    # Test 5: from_instruction works
    qc = QuantumCircuit(1)
    qc.h(0)
    sv = Statevector.from_instruction(qc)
    expected = Statevector.from_label('+')
    fid = state_fidelity(sv, expected)
    assert np.isclose(fid, 1.0), "Test 5 failed: H|0⟩ should equal |+⟩"
    
    print("✅ All Statevector tests passed!")

test_statevector()

In [ ]:
# ============================================================
# DensityMatrix - BASIC USAGE
# ============================================================

# Create from label (pure state)
rho_0 = DensityMatrix.from_label('0')
print("DensityMatrix from label |0⟩:")
print(rho_0.data)
print(f"Purity: {rho_0.purity()}")  # Should be 1.0 (pure)

# Create from Statevector
sv_plus = Statevector.from_label('+')
rho_plus = DensityMatrix(sv_plus)
print(f"\nDensityMatrix from |+⟩:")
print(np.round(rho_plus.data, 3))
print(f"Purity: {rho_plus.purity()}")  # Should be 1.0 (pure)

# Create maximally mixed state (ONLY DensityMatrix can do this!)
mixed = DensityMatrix(np.eye(2) / 2)
print(f"\nMaximally mixed state:")
print(mixed.data)
print(f"Purity: {mixed.purity()}")  # Should be 0.5 (mixed)

assert np.isclose(rho_0.purity(), 1.0), "Pure state purity should be 1"
assert np.isclose(mixed.purity(), 0.5), "Maximally mixed purity should be 0.5"
print("\n✅ DensityMatrix basic usage verified")

In [ ]:
# ============================================================
# DensityMatrix - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Create from various sources
print("[1] Different creation methods:")
print("-" * 40)
# From label
dm_label = DensityMatrix.from_label('+')
print(f"From label |+⟩: purity = {dm_label.purity():.2f}")

# From circuit
qc = QuantumCircuit(1)
qc.h(0)
dm_circuit = DensityMatrix.from_instruction(qc)
print(f"From circuit H|0⟩: purity = {dm_circuit.purity():.2f}")

# From numpy array (mixed state)
mixed_array = np.array([[0.7, 0.1], [0.1, 0.3]])  # Valid density matrix
dm_array = DensityMatrix(mixed_array)
print(f"From array: purity = {dm_array.purity():.2f}")

# Variation 2: Evolve through channel
print("\n[2] Evolve through quantum channel:")
print("-" * 40)
rho = DensityMatrix.from_label('0')
x_channel = Kraus(Operator(XGate()))
rho_evolved = rho.evolve(x_channel)
print(f"|0⟩ evolved by X channel:")
print(np.round(rho_evolved.data, 3))

# Variation 3: Tensor product
print("\n[3] Tensor product:")
print("-" * 40)
rho1 = DensityMatrix.from_label('0')
rho2 = DensityMatrix.from_label('+')
rho_tensor = rho1.tensor(rho2)
print(f"|0⟩⟨0| ⊗ |+⟩⟨+| has {rho_tensor.num_qubits} qubits")
print(f"Purity: {rho_tensor.purity():.2f}")

# Variation 4: Partial trace
print("\n[4] Partial trace (tracing out subsystems):")
print("-" * 40)
# Create Bell state density matrix
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
rho_bell = DensityMatrix.from_instruction(qc)
print(f"Bell state purity: {rho_bell.purity():.2f}")

# Trace out qubit 1 (leaves qubit 0)
rho_reduced = partial_trace(rho_bell, [1])
print(f"After tracing out qubit 1:")
print(np.round(rho_reduced.data, 3))
print(f"Reduced state purity: {rho_reduced.purity():.2f}")
print("Note: Maximally mixed! (Entanglement signature)")

print("\n✅ DensityMatrix variations demonstrated")

In [ ]:
# ============================================================
# DensityMatrix - VERIFICATION PATTERN
# ============================================================

def test_density_matrix():
    """Test suite for DensityMatrix class."""
    
    # Test 1: Pure state has purity 1
    rho = DensityMatrix.from_label('0')
    assert np.isclose(rho.purity(), 1.0), "Test 1 failed: pure state purity should be 1"
    
    # Test 2: Maximally mixed has purity 1/d
    mixed = DensityMatrix(np.eye(2) / 2)
    assert np.isclose(mixed.purity(), 0.5), "Test 2 failed: maximally mixed purity should be 0.5"
    
    # Test 3: Trace equals 1
    rho = DensityMatrix.from_label('+')
    trace = np.trace(rho.data)
    assert np.isclose(trace, 1.0), "Test 3 failed: trace should be 1"
    
    # Test 4: Hermitian (rho = rho†)
    rho = DensityMatrix.from_label('+-')
    assert np.allclose(rho.data, rho.data.conj().T), "Test 4 failed: should be Hermitian"
    
    # Test 5: Positive semi-definite (all eigenvalues ≥ 0)
    rho = DensityMatrix(np.eye(2) / 2)
    eigenvalues = np.linalg.eigvalsh(rho.data)
    assert all(ev >= -1e-10 for ev in eigenvalues), "Test 5 failed: should be positive semi-definite"
    
    # Test 6: Partial trace of Bell state gives maximally mixed
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    rho_bell = DensityMatrix.from_instruction(qc)
    rho_reduced = partial_trace(rho_bell, [1])
    assert np.isclose(rho_reduced.purity(), 0.5), "Test 6 failed: reduced Bell state should be maximally mixed"
    
    print("✅ All DensityMatrix tests passed!")

test_density_matrix()

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Superposition ≠ Mixture!
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Superposition is NOT the same as Mixture!")
print("=" * 50)

# |+⟩ = superposition (PURE state)
sv_plus = Statevector.from_label('+')
rho_plus = DensityMatrix(sv_plus)

# 50/50 mixture of |0⟩ and |1⟩ (MIXED state)
rho_mixed = DensityMatrix(np.array([[0.5, 0], [0, 0.5]]))

print("\n|+⟩ (superposition) - PURE STATE:")
print(f"Matrix:\n{np.round(rho_plus.data, 3)}")
print(f"Purity: {rho_plus.purity()}")
print("Note: Off-diagonal elements show coherence!")

print("\n50/50 mixture - MIXED STATE:")
print(f"Matrix:\n{rho_mixed.data}")
print(f"Purity: {rho_mixed.purity()}")
print("Note: No off-diagonal elements (no coherence)")

print("\n❌ WRONG: Using Statevector for classical mixture")
print("   Statevector CANNOT represent mixed states!")

print("\n✅ CORRECT: Use DensityMatrix for mixed states")
print("   Check purity: Pure=1, Mixed<1")

print("\n💡 Remember: DM for Dirty/Mixed, SV for Single/Pure (See README)")

---

## `state_fidelity()` and `process_fidelity()`

### Signature
```python
state_fidelity(state1, state2, validate=True)
process_fidelity(channel, target=None, ...)
average_gate_fidelity(channel, target=None, ...)
```

### Parameters
| Function | Parameter | Type | Description |
|----------|-----------|------|-------------|
| `state_fidelity` | `state1, state2` | `Statevector`, `DensityMatrix` | States to compare |
| `process_fidelity` | `channel, target` | `Operator`, `QuantumChannel` | Processes to compare |
| `average_gate_fidelity` | `channel, target` | `Operator`, `QuantumChannel` | Gate quality metric |

### Returns
`float` - Fidelity value in range **[0, 1]**

### See Also
- README Section: [Fidelity Measures](./README.md#-topic-94-fidelity-measures)

In [ ]:
# ============================================================
# state_fidelity() - BASIC USAGE
# ============================================================

# Identical states - Fidelity = 1
state1 = Statevector.from_label('0')
state2 = Statevector.from_label('0')
fid_identical = state_fidelity(state1, state2)
print(f"Fidelity(|0⟩, |0⟩) = {fid_identical}")

# Orthogonal states - Fidelity = 0
state_0 = Statevector.from_label('0')
state_1 = Statevector.from_label('1')
fid_orthogonal = state_fidelity(state_0, state_1)
print(f"Fidelity(|0⟩, |1⟩) = {fid_orthogonal}")

# Partially overlapping - Fidelity = 0.5
state_0 = Statevector.from_label('0')
state_plus = Statevector.from_label('+')
fid_partial = state_fidelity(state_0, state_plus)
print(f"Fidelity(|0⟩, |+⟩) = {fid_partial}")
print(f"Expected: |⟨0|+⟩|² = |1/√2|² = 0.5")

# Verify all fidelities are in [0, 1]
assert 0 <= fid_identical <= 1, "Fidelity must be in [0,1]"
assert 0 <= fid_orthogonal <= 1, "Fidelity must be in [0,1]"
assert 0 <= fid_partial <= 1, "Fidelity must be in [0,1]"
print("\n✅ All fidelities in [0, 1] range")

In [ ]:
# ============================================================
# process_fidelity() - BASIC USAGE
# ============================================================

# Identical operations - Fidelity = 1
op_x1 = Operator(XGate())
op_x2 = Operator(XGate())
proc_fid = process_fidelity(op_x1, op_x2)
print(f"Process fidelity(X, X) = {proc_fid}")

# Different operations
op_x = Operator(XGate())
op_z = Operator(ZGate())
proc_fid_xz = process_fidelity(op_x, op_z)
print(f"Process fidelity(X, Z) = {proc_fid_xz}")

# Average gate fidelity (standard metric)
avg_fid = average_gate_fidelity(op_x, op_x)
print(f"\nAverage gate fidelity(X, X) = {avg_fid}")

# Verify ranges
assert 0 <= proc_fid <= 1, "Process fidelity must be in [0,1]"
assert 0 <= avg_fid <= 1, "Avg gate fidelity must be in [0,1]"
print("\n✅ All fidelities verified in [0, 1]")

In [ ]:
# ============================================================
# Fidelity - VERIFICATION PATTERN
# ============================================================

def test_fidelity():
    """Test suite for fidelity functions."""
    
    # Test 1: Identical states have fidelity 1
    s1 = Statevector.from_label('00')
    fid = state_fidelity(s1, s1)
    assert np.isclose(fid, 1.0), "Test 1 failed: identical should be 1"
    
    # Test 2: Orthogonal states have fidelity 0
    s0 = Statevector.from_label('0')
    s1 = Statevector.from_label('1')
    fid = state_fidelity(s0, s1)
    assert np.isclose(fid, 0.0), "Test 2 failed: orthogonal should be 0"
    
    # Test 3: |0⟩ and |+⟩ have fidelity 0.5
    s0 = Statevector.from_label('0')
    splus = Statevector.from_label('+')
    fid = state_fidelity(s0, splus)
    assert np.isclose(fid, 0.5), "Test 3 failed: |0⟩,|+⟩ should be 0.5"
    
    # Test 4: Fidelity is symmetric
    s1 = Statevector.from_label('+')
    s2 = Statevector.from_label('0')
    assert np.isclose(state_fidelity(s1, s2), state_fidelity(s2, s1)), "Test 4 failed: should be symmetric"
    
    print("✅ All fidelity tests passed!")

test_fidelity()

---

## Quantum Channels: `Kraus`, `SuperOp`, `Choi`

### Signatures
```python
Kraus(data, input_dims=None, output_dims=None)
SuperOp(data, input_dims=None, output_dims=None)
Choi(data, input_dims=None, output_dims=None)
```

### When to Use Each
| Representation | Best For | Mnemonic |
|----------------|----------|----------|
| `Kraus` | Physics intuition | **K**now the physics |
| `SuperOp` | Mathematical manipulation | **S**olve mathematically |
| `Choi` | Tomography, property checks | **C**heck properties |

### See Also
- README Section: [Quantum Channels](./README.md#-topic-95-quantum-channels)

In [ ]:
# ============================================================
# Quantum Channels - BASIC USAGE
# ============================================================

# Create channels from unitary
x_op = Operator(XGate())

# Convert to different representations
kraus = Kraus(x_op)
superop = SuperOp(x_op)
choi = Choi(x_op)

print("Channel representations from X gate:")
print(f"Kraus operators: {len(kraus.data)} operator(s)")
print(f"SuperOp dimensions: {superop.dim}")
print(f"Choi matrix shape: {choi.data.shape}")

# Apply channel to state
rho = DensityMatrix.from_label('0')
rho_after = rho.evolve(kraus)
print(f"\n|0⟩ after X channel: {rho_after}")

# Verify X flips |0⟩ to |1⟩
expected = DensityMatrix.from_label('1')
fid = state_fidelity(rho_after, expected)
assert np.isclose(fid, 1.0), "X should flip |0⟩ to |1⟩"
print("✅ Channel application verified")

In [ ]:
# ============================================================
# Quantum Channels - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Create depolarizing channel
print("[1] Depolarizing channel:")
print("-" * 40)
def make_depolarizing(p):
    """Create depolarizing channel with error probability p."""
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    kraus_ops = [
        np.sqrt(1 - 3*p/4) * I,
        np.sqrt(p/4) * X,
        np.sqrt(p/4) * Y,
        np.sqrt(p/4) * Z
    ]
    return Kraus(kraus_ops)

depol = make_depolarizing(0.1)
print(f"Depolarizing channel with p=0.1: {len(depol.data)} Kraus operators")

# Variation 2: Convert between representations
print("\n[2] Convert between representations:")
print("-" * 40)
# Start with Kraus
kraus = Kraus(Operator(HGate()))
print(f"Kraus: {len(kraus.data)} operators")

# Convert to SuperOp
superop = SuperOp(kraus)
print(f"SuperOp: {superop.dim} dimensions")

# Convert to Choi
choi = Choi(kraus)
print(f"Choi: {choi.data.shape} matrix")

# Convert back to Kraus
kraus_back = Kraus(superop)
print(f"Kraus (from SuperOp): {len(kraus_back.data)} operators")

# Variation 3: Compose channels
print("\n[3] Compose channels:")
print("-" * 40)
x_channel = Kraus(Operator(XGate()))
z_channel = Kraus(Operator(ZGate()))
xz_channel = x_channel.compose(z_channel)  # Z first, then X
print(f"X ∘ Z channel (Z first, then X): {len(xz_channel.data)} operators")

# Variation 4: Apply noisy channel
print("\n[4] Effect of depolarizing noise:")
print("-" * 40)
rho_pure = DensityMatrix.from_label('+')
print(f"Initial |+⟩ purity: {rho_pure.purity():.3f}")

for p in [0.0, 0.1, 0.5, 1.0]:
    depol = make_depolarizing(p)
    rho_noisy = rho_pure.evolve(depol)
    print(f"After depolarizing p={p}: purity={rho_noisy.purity():.3f}")

print("\n✅ Quantum channel variations demonstrated")

In [ ]:
# ============================================================
# Quantum Channels - VERIFICATION PATTERN
# ============================================================

def test_quantum_channels():
    """Test suite for quantum channel classes."""
    
    # Test 1: Identity channel preserves state
    id_channel = Kraus([np.eye(2)])
    rho = DensityMatrix.from_label('+')
    rho_after = rho.evolve(id_channel)
    fid = state_fidelity(rho, rho_after)
    assert np.isclose(fid, 1.0), "Test 1 failed: identity should preserve state"
    
    # Test 2: X channel flips |0⟩ to |1⟩
    x_channel = Kraus(Operator(XGate()))
    rho_0 = DensityMatrix.from_label('0')
    rho_1 = DensityMatrix.from_label('1')
    rho_after = rho_0.evolve(x_channel)
    fid = state_fidelity(rho_after, rho_1)
    assert np.isclose(fid, 1.0), "Test 2 failed: X should flip |0⟩ to |1⟩"
    
    # Test 3: Conversions preserve behavior
    h_op = Operator(HGate())
    kraus = Kraus(h_op)
    superop = SuperOp(h_op)
    choi = Choi(h_op)
    
    rho = DensityMatrix.from_label('0')
    rho_k = rho.evolve(kraus)
    rho_s = rho.evolve(superop)
    rho_c = rho.evolve(choi)
    
    assert np.isclose(state_fidelity(rho_k, rho_s), 1.0), "Test 3a failed: Kraus != SuperOp"
    assert np.isclose(state_fidelity(rho_k, rho_c), 1.0), "Test 3b failed: Kraus != Choi"
    
    # Test 4: Depolarizing reduces purity
    def depolarizing(p):
        I, X = np.eye(2), np.array([[0, 1], [1, 0]])
        Y, Z = np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])
        return Kraus([np.sqrt(1-3*p/4)*I, np.sqrt(p/4)*X, np.sqrt(p/4)*Y, np.sqrt(p/4)*Z])
    
    rho = DensityMatrix.from_label('+')
    rho_noisy = rho.evolve(depolarizing(0.5))
    assert rho_noisy.purity() < rho.purity(), "Test 4 failed: depolarizing should reduce purity"
    
    # Test 5: Trace preserving (output trace = 1)
    x_channel = Kraus(Operator(XGate()))
    rho = DensityMatrix.from_label('0')
    rho_after = rho.evolve(x_channel)
    trace = np.trace(rho_after.data)
    assert np.isclose(trace, 1.0), "Test 5 failed: channel should preserve trace"
    
    print("✅ All quantum channel tests passed!")

test_quantum_channels()

In [ ]:
# ============================================================
# COMPARISON: Statevector vs DensityMatrix
# ============================================================

print("=" * 60)
print("COMPARING: Statevector vs DensityMatrix")
print("=" * 60)

# Create same state with both
qc = QuantumCircuit(1)
qc.h(0)

sv = Statevector.from_instruction(qc)
dm = DensityMatrix.from_instruction(qc)

print("\nStatevector |+⟩:")
print(f"  Data: {sv.data}")
print(f"  Size: {len(sv.data)} elements")

print("\nDensityMatrix |+⟩⟨+|:")
print(f"  Data:\n{np.round(dm.data, 3)}")
print(f"  Size: {dm.data.shape} elements")
print(f"  Purity: {dm.purity()}")

print("\nKey difference:")
print("  Statevector: |ψ⟩ = [α, β]")
print("  DensityMatrix: ρ = |ψ⟩⟨ψ| (matrix)")

print("\n✅ DensityMatrix can also represent MIXED states (purity < 1)")

---

## Randomized Benchmarking (Conceptual Code Patterns)

### Overview
Randomized Benchmarking (RB) measures gate errors by applying random Clifford sequences.

**Note**: RB requires actual quantum hardware via `qiskit-experiments`. 
The code patterns below show the API structure for exam preparation.

### Key Concepts
| Term | Meaning | Typical Values |
|------|---------|----------------|
| **EPC** | Error Per Clifford | 0.001-0.01 (good) |
| **SPAM-free** | Independent of state prep/measurement | Key RB benefit |
| **Sequence length** | Number of Cliffords applied | [1, 10, 50, 100, 200] |

### See Also
- README Section: [Randomized Benchmarking](./README.md#-topic-96-randomized-benchmarking)

In [ ]:
# ============================================================
# Randomized Benchmarking - CODE PATTERNS (Exam Reference)
# ============================================================
# NOTE: These patterns require qiskit-experiments and hardware
# This cell demonstrates the API structure for exam preparation

print("Randomized Benchmarking - API Reference")
print("=" * 50)

# Pattern 1: StandardRB Setup (conceptual - requires qiskit-experiments)
print("\n[1] StandardRB Setup Pattern:")
print("-" * 40)
rb_setup_code = '''
from qiskit_experiments.library import StandardRB
from qiskit_ibm_runtime import QiskitRuntimeService

# Connect to IBM Quantum
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

# Create RB experiment
rb_exp = StandardRB(
    physical_qubits=[0],           # Which qubit(s) to benchmark
    lengths=[1, 10, 20, 50, 100],  # Sequence lengths
    num_samples=10,                # Circuits per length
    seed=42                        # Reproducibility
)

# Run experiment
rb_data = rb_exp.run(backend).block_for_results()

# Extract Error Per Clifford
epc = rb_data.analysis_results('EPC').value
print(f"Error per Clifford: {epc:.4f}")
'''
print(rb_setup_code)

# Pattern 2: RB uses random Cliffords (we CAN demonstrate this!)
print("\n[2] RB uses Random Cliffords (demonstrable):")
print("-" * 40)
from qiskit.quantum_info import random_clifford, Clifford

# Generate random Clifford sequence
sequence_length = 5
cliffords = [random_clifford(1, seed=i) for i in range(sequence_length)]
print(f"Generated {sequence_length} random Cliffords")

# Compose them all
composed = cliffords[0]
for c in cliffords[1:]:
    composed = composed.compose(c)
print(f"Composed Clifford: {composed.num_qubits} qubit(s)")

# Find the inverse (key RB concept!)
# The inverse should return state to |0⟩ if no errors
inverse = composed.adjoint()
print("Inverse Clifford computed")

# Verify: composed ∘ inverse = Identity
identity_check = composed.compose(inverse)
identity_cliff = Clifford(QuantumCircuit(1))  # Identity Clifford
print(f"Composed with inverse gives identity: {identity_check == identity_cliff}")

print("\n✅ Random Clifford generation demonstrated")

---

## 🎯 Code Challenges

Test your understanding of quantum information tools!

In [ ]:
# ============================================================
# CHALLENGE 1: Verify Circuit Equivalence
# ============================================================
# Task: Check if these two circuits implement the same operation
# Hint: Use Operator.equiv()
# ============================================================

def challenge_1():
    """
    Two circuits that may or may not be equivalent.
    Return True if equivalent, False otherwise.
    """
    qc1 = QuantumCircuit(1)
    qc1.h(0)
    qc1.z(0)
    qc1.h(0)
    
    qc2 = QuantumCircuit(1)
    qc2.x(0)
    
    # YOUR CODE HERE
    # Check if qc1 and qc2 implement the same operation
    op1 = Operator(qc1)
    op2 = Operator(qc2)
    
    return op1.equiv(op2)

# Test your solution
result = challenge_1()
print(f"Are the circuits equivalent? {result}")

if result == True:
    print("✅ Challenge 1 PASSED! HZH = X")
else:
    print("❌ Challenge 1 FAILED - try again")

In [ ]:
# ============================================================
# CHALLENGE 2: Calculate State Fidelity
# ============================================================
# Task: Create a Bell state and calculate its fidelity with |00⟩
# Expected: Fidelity should be 0.5 (Bell state has 50% overlap)
# ============================================================

def challenge_2():
    """
    Create Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2
    Return fidelity with |00⟩
    """
    # YOUR CODE HERE
    # 1. Create Bell state circuit
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    
    # 2. Get statevector
    bell_state = Statevector.from_instruction(qc)
    
    # 3. Create |00⟩
    state_00 = Statevector.from_label('00')
    
    # 4. Calculate fidelity
    fid = state_fidelity(bell_state, state_00)
    
    return fid

# Test your solution
fid = challenge_2()
print(f"Fidelity(Bell, |00⟩) = {fid}")

if np.isclose(fid, 0.5):
    print("✅ Challenge 2 PASSED!")
else:
    print(f"❌ Challenge 2 FAILED - expected 0.5, got {fid}")

In [ ]:
# ============================================================
# CHALLENGE 3: Distinguish Pure from Mixed State
# ============================================================
# Task: Given a DensityMatrix, determine if it's pure or mixed
# Hint: Check purity - Pure states have purity = 1
# ============================================================

def challenge_3(rho):
    """
    Determine if a density matrix represents a pure or mixed state.
    
    Args:
        rho: DensityMatrix object
        
    Returns:
        str: "pure" or "mixed"
    """
    # YOUR CODE HERE
    purity = rho.purity()
    
    if np.isclose(purity, 1.0):
        return "pure"
    else:
        return "mixed"

# Test your solution
# Test case 1: Pure state |0⟩
rho_pure = DensityMatrix.from_label('0')
result1 = challenge_3(rho_pure)
print(f"Test 1 - |0⟩ state: {result1} (purity={rho_pure.purity():.2f})")

# Test case 2: Maximally mixed state
rho_mixed = DensityMatrix(np.eye(2) / 2)
result2 = challenge_3(rho_mixed)
print(f"Test 2 - Mixed state: {result2} (purity={rho_mixed.purity():.2f})")

# Verify
if result1 == "pure" and result2 == "mixed":
    print("✅ Challenge 3 PASSED!")
else:
    print("❌ Challenge 3 FAILED - check your logic")

---

## 📚 Resources

### Conceptual Background
- [README.md](./README.md) - Full explanations, traps, mnemonics

### Key Mnemonics (See README for details)
- **HSCP - No T!** - Clifford gates
- **EQUIV for EQUIValent** - Use .equiv() not ==
- **Fidelity = Faithful to [0,1]** - Range always 0-1
- **DM for Dirty/Mixed, SV for Single/Pure** - When to use each

### API Documentation
- [Qiskit Quantum Information](https://docs.quantum.ibm.com/api/qiskit/quantum_info)

---

**Notebook verified with Qiskit 1.x** | Last updated: December 2025